#Model 2

edit the data directory in block 4

run code blocks in order to train model2
(may cause out of memory error especially when run with model 1)

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader
import imageio.v3 as imageio
import torchvision.transforms.v2 as transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
n_epochs = 6
n_batch_size = 10
n_display_step = 10
n_learning_rate = 0.0001

output_col = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']
output_col_test = ['X4', 'X11', 'X18', 'X26', 'X50', 'X3112']

datapath = '/kaggle/input/cs-480-2024-spring/data/' # edit to directory with data
train = pd.read_csv(datapath + 'train.csv')
train['path'] = [datapath + 'train_images/'+str(int(e))+'.jpeg' for e in train['id'].values]
train['jb'] = train['path'].apply(lambda fp: open(fp, 'rb').read())

test = pd.read_csv(datapath + 'test.csv')
test['path'] = [datapath + 'test_images/'+str(int(e))+'.jpeg' for e in test['id'].values]
test['jb'] = test['path'].apply(lambda fp: open(fp, 'rb').read())


# remove outliers
for column in output_col:
    lower_quantile = train[column].quantile(0.005)
    upper_quantile = train[column].quantile(0.985)
    train = train[(train[column] >= lower_quantile) & (train[column] <= upper_quantile)]

print(train.shape)

log_feat = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']

y_train_log = np.zeros_like(train[output_col])

for i, tar in enumerate(output_col):
    col = train[tar].values
    if tar in log_feat:
        col = np.log10(col)
    y_train_log[:, i] = col


SCALER = StandardScaler()
y_train_norm = SCALER.fit_transform(y_train_log)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_im, data_out, transform=None, mode='m1train'):
        self.data_im = data_im
        self.data_out = data_out
        self.transform = transform
        self.mode = mode

    def __getitem__(self, index):
        x = self.transform(imageio.imread(self.data_im[index]))
        y = self.data_out[index]
        return x, y

    def __len__(self):
        return len(self.data_im)

In [ ]:
test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((384, 384)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)
train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((384, 384)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter((0.9,1.1), (0.9,1.1)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

trainx, valx,trainy, valy = train_test_split(train, y_train_norm, test_size=2000)

train_dataset = MyDataset(trainx['jb'].values, trainy, train_transform, 'm2train')
val_dataset = MyDataset(valx['jb'].values, valy, test_transform, 'm2train')
test_dataset = MyDataset(test['jb'].values, test['id'].values, test_transform, 'm2test')

train_dataloader = DataLoader(train_dataset, batch_size=n_batch_size, drop_last=True)

val_dataloader = DataLoader(val_dataset, batch_size=n_batch_size, drop_last=False)

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.swin = timm.create_model(
                'swin_large_patch4_window12_384.ms_in22k_ft_in1k',
                num_classes=6,
                pretrained=True)

    def forward(self, input):
        return self.swin(input)

model2 = Model2()
model2 = model2.to(device)

In [ ]:
def lr_sc(optimizer):
    return torch.optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=1e-4,
        total_steps=len(train_dataset)//n_batch_size * n_epochs + 1,
        pct_start=0.1,
        anneal_strategy='cos',
        div_factor=1e1,
        final_div_factor=1e1,
    )

optimizer = optim.AdamW(model2.parameters(), lr=n_learning_rate, weight_decay=0.01)
criterion = nn.SmoothL1Loss()
scheduler = lr_sc(optimizer)

In [ ]:
def get_r2(model, loader, mode):
    model.eval()
    n_samples = 0
    n_correct = 0
    res = []
    tar = []

    with torch.no_grad():
        test_loss = []
        for step, (x, target) in enumerate(loader):
            x=x.to(device)
            target = target.to(device)
            output = model(x)
            res.extend(output.detach().cpu())
            tar.extend(target.detach().cpu())
            loss = criterion(output, target)
            if step % n_display_step == 0:
                print('\rloss: ', loss.item(), end='')
    tar = np.array(tar)
    res = np.array(res)
    print([r2_score(tar[:,i],res[:,i]) for i in range(6)])
    return [r2_score(tar[:,i],res[:,i]) for i in range(6)]

# function to train the net
def train_model(model, criterion, optimizer, epochs, train_loader, val_loader):
    for epoch in range(epochs):
        model.train()
        for step, (x, target) in enumerate(train_loader):
            x = x.to(device)
            target = target.to(device)
            output = model(x)
            print('\r'+str(step),end='')
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

            if step % n_display_step == 0:
                print("\rEpoch {:2d} Step {:4d} Loss {:.4f}".format(epoch, step, loss.detach().item()), f'{scheduler.get_last_lr()[0]:.2e}', end='')
        val_acc = get_r2(model, val_loader, 'test')
        print("Epoch {:2d} Loss {:.4f} Accuracy (Train | Test) {:.4f} {:.4f}".format(epoch, loss.detach().item(), val_acc))


train_model(model2, criterion, optimizer, n_epochs, train_dataloader, val_dataloader)

In [ ]:
model2.eval()
test_output=[]
with torch.no_grad():
    for step, (x, id) in enumerate(test_dataset):
        x = x.unsqueeze(0).to(device)
        output = model2(x).detach().cpu()

        output = SCALER.inverse_transform(output).squeeze(0)

        output = 10**output
        test_output.append(np.append(np.array(id),output))

df = pd.DataFrame(test_output, columns=['id']+output_col_test)
df['id'] = df['id'].astype(int)
df.to_csv('submissionm2.csv', index=False)
